In [53]:
#Create table
import sqlite3


def insertDB(db_name, data_name, data, date='', filename=''):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    
    if data_name == "house_raw":
        #print("Cars data filename is ", filename)
        # Create table
        #https://www.etuovi.com/kohde/1244266?sc=LRnomun_nodistr&pos=11,Tapulikaupunki Helsinki,Palokuja 4,Kerrostalo,3h+k+kph+vh+lasitettu parveke,"65,0 m²",169 000 €,1981
        #link, location, address, house type, rooms, size, price, year
        c.execute("""CREATE TABLE IF NOT EXISTS house_raw (ind integer, link text, location text, address text, house_type text, rooms text, size text, price text, year integer, insert_date date, batchname varchar(50))""")
        #print("Table created")
        count=conn.execute("select count(*) from house_raw")
        #print("Rows already in DB: ", count)
        for cou in count:
            count=cou[0]
        # Larger example that inserts many records at a time
        data_insert=[]
        for d in data:
            #Generate the rows
            count = count+1
            data_insert.append([count, d[0], d[1], d[2], d[3], d[4], d[5], d[6], d[7], date, filename])
        c.executemany('INSERT INTO house_raw VALUES (?,?,?,?,?,?,?,?,?,?,?)', data_insert)
    

    # Save (commit) the changes
    conn.commit()
    # Close the connection
    conn.close()
        
def createTable(columns, data_name):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    create_string="CREATE TABLE IF NOT EXISTS " + data_name + "(" + columns + ")"
    print(create_string)
    c.execute(create_string)
    conn.commit()
    conn.close()
    
def getHouses():
    conn = sqlite3.connect(db_name)
    for row in conn.execute("select * from house_raw"):
        print (row)
    conn.close()
        
def get(db_name,data_name):
    conn = sqlite3.connect(db_name)
    for row in conn.execute("select * from "+data_name):
        print (row)
    conn.close()
        
def getColumn(data_name, column, db_name):
    conn = sqlite3.connect(db_name)
    l=[]
    for row in conn.execute("select "+column+" from "+data_name):
        l.append (row)
    conn.close()
    return l
    
def deleteHouses(db_name):
    conn = sqlite3.connect(db_name)
    conn.execute("delete from house_raw")
    conn.commit()
    conn.close()
    
def dropHouses(db_name):
    conn = sqlite3.connect(db_name)
    conn.execute("drop table house_raw")
    conn.commit()
    conn.close()
    
def delete(db_name,data_name):
    conn = sqlite3.connect(db_name)
    conn.execute("delete from "+ data_name)
    conn.commit()
    conn.close()


In [59]:
import os,csv
#Generate database based on csv. Insert matching csv:s with function from first cell.

def getFilesInserted():
    data_name="house_raw"
    column="DISTINCT batchname"
    db_name = "house.db"
    try:
        lines=getColumn(data_name, column, db_name)
    except:
        print("Error when getting row count, assuming row count to be 0")
        return []
    res = []
    for l in lines:
        #print(l[0])
        res.append(l[0])
    return res

def insertCsvToDb():
    db_name="house.db"
    data_name="house_raw"
    dir_name="house data"
    files_inserted=getFilesInserted()
    #print(files_inserted)
    failed=0
    data_dir=os.listdir('./'+dir_name)
    data_dir.sort()
    #print(data_dir)
    for filename in data_dir:
        if filename not in files_inserted and filename.endswith(".csv"):
            date = filename.split(" ")[0]
            print("Inserting " + filename)
            results = []
            with open('./'+dir_name+"/"+filename) as csvfile:
                reader = csv.reader(csvfile) # change contents to floats
                try:
                    for row in reader: # each row is a list
                        try:
                            if len(row)>0:
                                results.append(row)
                        except:
                            failed=failed+1
                            #raise
                    insertDB(db_name, data_name, results, date, filename)
                    #print(filename)
                except Exception as e:
                    print("exception "+str(e))
                    #raise
        elif not filename.endswith(".csv"):
            #print(filename, " skipped")
            1
        else:
            print(filename, " already inserted")

    print("Failed "+str(failed))

db_name="house.db"
test=0
if test:
    print("Running test run - deleting the database")
    try:
        dropHouses(db_name)
        print("Deleted")
    except:
        print("Not deleted")

insertCsvToDb()

2019-01-26 etuovi.csv  already inserted
2019-01-29 etuovi.csv  already inserted
2019-02-09 etuovi.csv  already inserted
Failed 0


In [57]:
#getHouses()